In [ ]:
import pandas as pd
import numpy as np
import networkx as nx
from typing import TypedDict, Dict, List, Optional, Any
from datetime import datetime, date, timedelta
from collections import defaultdict
import json
from langgraph.graph import StateGraph, END
from api_config import get_openai_client




In [ ]:
CANONICAL_REGIONS = [
    "Вінницька", "Волинська", "Дніпропетровська", "Донецька",
    "Житомирська", "Закарпатська", "Запорізька",
    "Івано-Франківська", "Київська", "Кіровоградська",
    "Луганська", "Львівська", "Миколаївська", "Одеська",
    "Полтавська", "Рівненська", "Сумська", "Тернопільська",
    "Харківська", "Херсонська", "Хмельницька", "Черкаська",
    "Чернівецька", "Чернігівська",
    "Київ", "м. Севастополь", "АР Крим"
]

REGION_COORDS = {
    "Вінницька": (49.233, 28.467),
    "Волинська": (50.733, 25.317),
    "Дніпропетровська": (48.467, 35.033),
    "Донецька": (48.000, 37.800),
    "Житомирська": (50.250, 28.650),
    "Закарпатська": (48.617, 22.283),
    "Запорізька": (47.833, 35.133),
    "Івано-Франківська": (48.917, 24.700),
    "Київська": (50.450, 30.517),
    "Київ": (50.450, 30.517),
    "Кіровоградська": (48.500, 32.267),
    "Луганська": (48.567, 39.300),
    "Львівська": (49.833, 24.017),
    "Миколаївська": (46.967, 31.983),
    "Одеська": (46.467, 30.717),
    "Полтавська": (49.583, 34.550),
    "Рівненська": (50.617, 26.250),
    "Сумська": (50.900, 34.783),
    "Тернопільська": (49.550, 25.583),
    "Харківська": (49.983, 36.217),
    "Херсонська": (46.633, 32.600),
    "Хмельницька": (49.417, 26.967),
    "Черкаська": (49.433, 32.050),
    "Чернівецька": (48.283, 25.933),
    "Чернігівська": (51.483, 31.283),
    "АР Крим": (44.950, 34.100),
    "м. Севастополь": (44.600, 33.533),
}


In [ ]:
class CanonicalState(TypedDict):
    """
    Канонічний стан енергосистеми для кожного регіону та моменту часу
    """
    # Ідентифікація
    region: str
    timestamp: str  
    
    # Дані ризику (з dataset_preparation)
    air_raid_alert: int  # 0/1
    alert_intensity: float  # 0.0 - 1.0
    alert_duration_minutes: Optional[int]
    
    blackout_status: str  # "none", "planned", "unplanned", "probability"
    blackout_probability: float  # 0.0 - 1.0
    blackout_consumers: int
    blackout_settlements: int
    
    # Погодні дані
    weather_temp: Optional[float]
    weather_wind_speed: Optional[float]
    weather_wind_gusts: Optional[float]
    weather_precipitation: Optional[float]
    weather_snowfall: Optional[float]
    weather_cloud_cover: Optional[float]
    weather_pressure: Optional[float]
    weather_extreme_flag: bool
    
    # Енергетичні дані (з data_electricity)
    demand: Optional[float]
    net_generation: Optional[float]
    total_interchange: Optional[float]
    
    # Похідні ознаки
    demand_rolling_mean: Optional[float]
    demand_volatility: Optional[float]
    generation_rolling_mean: Optional[float]
    generation_volatility: Optional[float]
    
    # Метадані
    data_completeness: float  # 0.0 - 1.0 (якість даних)
    last_updated: str  # ISO timestamp
    source_priority: Dict[str, int]  # Пріоритет джерел даних


In [ ]:
class OrchestratorState(TypedDict):
    """
    Стан оркестратора - містить всю інформацію про процес синхронізації та обробки
    """
    # Вхідні дані з різних джерел
    risk_data: Dict[str, pd.DataFrame]  # Дані ризику по регіонах
    energy_data: Dict[str, pd.DataFrame]  # Енергетичні дані по регіонах
    
    # Канонічний стан (синхронізований)
    canonical_states: Dict[str, CanonicalState]  # key: f"{region}_{timestamp}"
    
    # Energy graph
    energy_graph: Optional[nx.Graph]  # Граф енергосистеми
    
    # Контекст для LLM
    llm_context: Optional[Dict[str, Any]]  # Зведений контекст для передачі LLM
    should_call_llm: bool  # Чи потрібно викликати LLM
    llm_response: Optional[str]  # Відповідь від LLM
    
    # Метадані
    sync_timestamp: str  # Час синхронізації
    regions_processed: List[str]  # Список оброблених регіонів
    sync_errors: List[str]  # Помилки синхронізації
    
    # Результат
    decision_required: bool  # Чи потрібне прийняття рішення
    critical_regions: List[str]  # Регіони з критичним станом


In [ ]:

def load_risk_data(file_path: str = None) -> pd.DataFrame:
    """
    Завантажує дані ризику з GNN_AGENT підготовки
    Очікує колонки: region (або ua_region), date (або timestamp_utc), 
    Alerts (або alert_active), isDamaged, temperature_mean (або temperature),
    wind_speed_max (або wind_speed), precipitation, snowfall, etc.
    """
    if file_path is None:
        # Спробуємо завантажити з підготовленого датасету GNN_AGENT
        try:
            from pathlib import Path
            data_dir = Path("data")
            # Можна використати готовий файл з GNN_AGENT після об'єднання з погодою
            # Або завантажити base_df після об'єднання з погодою
            df = pd.read_csv(data_dir / "combined_region_electricity.csv", parse_dates=["date"])
            # Нормалізуємо назву колонки регіону якщо потрібно
            if "ua_region" in df.columns and "region" not in df.columns:
                df = df.rename(columns={"ua_region": "region"})
            return df
        except FileNotFoundError:
            print("Файл не знайдено, використовуємо приклад даних")
            return create_sample_risk_data()
    
    df = pd.read_csv(file_path, parse_dates=["date"])
    # Нормалізуємо назву колонки регіону якщо потрібно
    if "ua_region" in df.columns and "region" not in df.columns:
        df = df.rename(columns={"ua_region": "region"})
    return df


def load_energy_data(file_path: str = None) -> pd.DataFrame:
    """
    Завантажує енергетичні дані з LOAD_AGENT_PREPARE
    Очікує колонки: ua_region (або region), timestamp_utc (або date),
    demand_ua_adj (або demand), generation_ua_adj (або net_generation),
    energy_deficit_adj (або total_interchange)
    """
    if file_path is None:
        # Завантажуємо з готового файлу LOAD_AGENT_PREPARE
        try:
            from pathlib import Path
            data_dir = Path("data")
            df = pd.read_csv(data_dir / "ready2use_region_electricity.csv", parse_dates=["timestamp_utc"])
            
            # Нормалізуємо назви колонок
            if "ua_region" in df.columns and "region" not in df.columns:
                df = df.rename(columns={"ua_region": "region"})
            
            # Створюємо колонку date з timestamp_utc
            if "date" not in df.columns and "timestamp_utc" in df.columns:
                df["date"] = pd.to_datetime(df["timestamp_utc"]).dt.date
            
            # Маппінг назв колонок до очікуваних
            if "demand_ua_adj" in df.columns and "demand" not in df.columns:
                df["demand"] = df["demand_ua_adj"]
            if "generation_ua_adj" in df.columns and "net_generation" not in df.columns:
                df["net_generation"] = df["generation_ua_adj"]
            if "energy_deficit_adj" in df.columns and "total_interchange" not in df.columns:
                # Дефіцит означає негативний interchange
                df["total_interchange"] = -df["energy_deficit_adj"]
            
            return df
        except FileNotFoundError:
            print("Файл не знайдено, використовуємо приклад даних")
            return create_sample_energy_data()
    
    df = pd.read_csv(file_path, parse_dates=["date"])
    # Нормалізуємо назви колонок якщо потрібно
    if "ua_region" in df.columns and "region" not in df.columns:
        df = df.rename(columns={"ua_region": "region"})
    if "demand_ua_adj" in df.columns and "demand" not in df.columns:
        df["demand"] = df["demand_ua_adj"]
    if "generation_ua_adj" in df.columns and "net_generation" not in df.columns:
        df["net_generation"] = df["generation_ua_adj"]
    return df


def create_sample_risk_data() -> pd.DataFrame:
    """Створює приклад даних ризику для тестування"""
    dates = pd.date_range(start="2024-01-01", end="2024-01-10", freq="D")
    regions = CANONICAL_REGIONS[:5]  # Перші 5 регіонів
    
    data = []
    for date in dates:
        for region in regions:
            data.append({
                "region": region,
                "date": date,
                "Alerts": np.random.randint(0, 3),
                "isDamaged": np.random.choice([True, False], p=[0.1, 0.9]),
                "temperature_mean": np.random.uniform(-5, 15),
                "wind_speed_max": np.random.uniform(5, 25),
                "precipitation": np.random.uniform(0, 20),
                "number_blackout_consumers": np.random.randint(0, 5000) if np.random.random() < 0.2 else 0
            })
    
    return pd.DataFrame(data)


def create_sample_energy_data() -> pd.DataFrame:
    """Створює приклад енергетичних даних для тестування"""
    dates = pd.date_range(start="2024-01-01", end="2024-01-10", freq="D")
    regions = CANONICAL_REGIONS[:5]
    
    data = []
    for date in dates:
        for region in regions:
            data.append({
                "region": region,
                "date": date,
                "consumption_kwh": np.random.uniform(10000, 30000),
                "demand": np.random.uniform(10000, 30000),
                "net_generation": np.random.uniform(8000, 28000),
                "total_interchange": np.random.uniform(-2000, 2000)
            })
    
    return pd.DataFrame(data)



In [ ]:
def synchronize_data(
    risk_df: pd.DataFrame,
    energy_df: pd.DataFrame,
    target_timestamp: Optional[datetime] = None
) -> Dict[str, CanonicalState]:
    """
    Синхронізує дані з різних джерел у канонічний стан
    
    Args:
        risk_df: DataFrame з даними ризику
        energy_df: DataFrame з енергетичними даними
        target_timestamp: Цільовий момент часу (за замовчуванням - останній доступний)
    
    Returns:
        Словник канонічних станів по регіонах
    """
    canonical_states = {}
    
    # Нормалізуємо формат дат та назв колонок
    # Для risk_df
    if "date" not in risk_df.columns and "timestamp_utc" in risk_df.columns:
        risk_df["date"] = pd.to_datetime(risk_df["timestamp_utc"]).dt.date
    else:
        risk_df["date"] = pd.to_datetime(risk_df["date"]).dt.date
    
    # Нормалізуємо назву колонки регіону в risk_df
    if "ua_region" in risk_df.columns and "region" not in risk_df.columns:
        risk_df["region"] = risk_df["ua_region"]
    
    # Для energy_df
    if "date" not in energy_df.columns and "timestamp_utc" in energy_df.columns:
        energy_df["date"] = pd.to_datetime(energy_df["timestamp_utc"]).dt.date
    else:
        energy_df["date"] = pd.to_datetime(energy_df["date"]).dt.date
    
    # Нормалізуємо назву колонки регіону в energy_df
    if "ua_region" in energy_df.columns and "region" not in energy_df.columns:
        energy_df["region"] = energy_df["ua_region"]
    
    # Визначаємо цільовий timestamp
    if target_timestamp is None:
        all_dates = set(risk_df["date"].unique()) | set(energy_df["date"].unique())
        target_date = max(all_dates) if all_dates else date.today()
    else:
        target_date = target_timestamp.date() if isinstance(target_timestamp, datetime) else target_timestamp
    
    # Обробляємо кожен регіон
    for region in CANONICAL_REGIONS:
        # Фільтруємо дані по регіону та даті
        risk_region = risk_df[
            (risk_df["region"] == region) & 
            (risk_df["date"] == target_date)
        ]
        energy_region = energy_df[
            (energy_df["region"] == region) & 
            (energy_df["date"] == target_date)
        ]
        
        # Створюємо канонічний стан
        state = create_canonical_state(region, target_date, risk_region, energy_region)
        key = f"{region}_{target_date.isoformat()}"
        canonical_states[key] = state
    
    return canonical_states


def create_canonical_state(
    region: str,
    timestamp: date,
    risk_data: pd.DataFrame,
    energy_data: pd.DataFrame
) -> CanonicalState:
    """
    Створює канонічний стан з даних ризику та енергетики
    """
    # Ініціалізуємо стан значеннями за замовчуванням
    state: CanonicalState = {
        "region": region,
        "timestamp": timestamp.isoformat(),
        "air_raid_alert": 0,
        "alert_intensity": 0.0,
        "alert_duration_minutes": None,
        "blackout_status": "none",
        "blackout_probability": 0.0,
        "blackout_consumers": 0,
        "blackout_settlements": 0,
        "weather_temp": None,
        "weather_wind_speed": None,
        "weather_wind_gusts": None,
        "weather_precipitation": None,
        "weather_snowfall": None,
        "weather_cloud_cover": None,
        "weather_pressure": None,
        "weather_extreme_flag": False,
        "demand": None,
        "net_generation": None,
        "total_interchange": None,
        "demand_rolling_mean": None,
        "demand_volatility": None,
        "generation_rolling_mean": None,
        "generation_volatility": None,
        "data_completeness": 0.0,
        "last_updated": datetime.now().isoformat(),
        "source_priority": {}
    }
    
    # Заповнюємо дані ризику
    if not risk_data.empty:
        row = risk_data.iloc[0]
        
        # Тривоги - обробляємо різні назви колонок (Alerts або alert_active)
        alerts_value = row.get("Alerts", row.get("alert_active", 0))
        if isinstance(alerts_value, bool):
            alerts_value = 1 if alerts_value else 0
        state["air_raid_alert"] = int(alerts_value > 0)
        state["alert_intensity"] = min(float(alerts_value) / 10.0, 1.0)
        
        # Відключення
        blackout_consumers = row.get("number_blackout_consumers", 0)
        if blackout_consumers > 0:
            state["blackout_status"] = "unplanned"
            state["blackout_consumers"] = int(blackout_consumers)
            state["blackout_probability"] = min(blackout_consumers / 10000.0, 1.0)
        
        # Погода - обробляємо різні назви колонок
        # temperature_mean (GNN) або temperature (LOAD)
        temp_value = row.get("temperature_mean", row.get("temperature", np.nan))
        state["weather_temp"] = float(temp_value) if pd.notna(temp_value) else None
        
        # wind_speed_max (GNN) або wind_speed (LOAD)
        wind_speed_value = row.get("wind_speed_max", row.get("wind_speed", np.nan))
        state["weather_wind_speed"] = float(wind_speed_value) if pd.notna(wind_speed_value) else None
        
        # wind_gusts_max (GNN) або wind_gusts (LOAD)
        wind_gusts_value = row.get("wind_gusts_max", row.get("wind_gusts", np.nan))
        state["weather_wind_gusts"] = float(wind_gusts_value) if pd.notna(wind_gusts_value) else None
        
        # precipitation
        precip_value = row.get("precipitation", np.nan)
        state["weather_precipitation"] = float(precip_value) if pd.notna(precip_value) else None
        
        # snowfall
        snow_value = row.get("snowfall", np.nan)
        state["weather_snowfall"] = float(snow_value) if pd.notna(snow_value) else None
        
        # cloud_cover_mean (GNN) або cloud_cover (LOAD)
        cloud_value = row.get("cloud_cover_mean", row.get("cloud_cover", np.nan))
        state["weather_cloud_cover"] = float(cloud_value) if pd.notna(cloud_value) else None
        
        # surface_pressure_mean (GNN) або surface_pressure (LOAD)
        pressure_value = row.get("surface_pressure_mean", row.get("surface_pressure", np.nan))
        state["weather_pressure"] = float(pressure_value) if pd.notna(pressure_value) else None
        
        # Екстремальні погодні умови
        if state["weather_wind_speed"] and state["weather_wind_speed"] > 25:
            state["weather_extreme_flag"] = True
        if state["weather_wind_gusts"] and state["weather_wind_gusts"] > 30:
            state["weather_extreme_flag"] = True
    
    # Заповнюємо енергетичні дані - обробляємо різні назви колонок
    if not energy_data.empty:
        row = energy_data.iloc[0]
        
        # demand або demand_ua_adj
        demand_value = row.get("demand", row.get("demand_ua_adj", np.nan))
        state["demand"] = float(demand_value) if pd.notna(demand_value) else None
        
        # net_generation або generation_ua_adj
        gen_value = row.get("net_generation", row.get("generation_ua_adj", np.nan))
        state["net_generation"] = float(gen_value) if pd.notna(gen_value) else None
        
        # total_interchange або energy_deficit_adj (з протилежним знаком)
        interchange_value = row.get("total_interchange", None)
        if interchange_value is None or pd.isna(interchange_value):
            # Якщо немає interchange, використовуємо energy_deficit як індикатор
            deficit_value = row.get("energy_deficit_adj", np.nan)
            if pd.notna(deficit_value):
                # Дефіцит означає негативний interchange
                state["total_interchange"] = -float(deficit_value)
            else:
                state["total_interchange"] = None
        else:
            state["total_interchange"] = float(interchange_value) if pd.notna(interchange_value) else None
    
    # Обчислюємо повноту даних
    fields = [
        state["air_raid_alert"] is not None,
        state["blackout_status"] != "none",
        state["weather_temp"] is not None,
        state["demand"] is not None,
        state["net_generation"] is not None
    ]
    state["data_completeness"] = sum(fields) / len(fields)
    
    return state

print("Функції синхронізації готові")


In [ ]:
def build_energy_graph(canonical_states: Dict[str, CanonicalState]) -> nx.Graph:
    """
    Будує граф енергосистеми на основі канонічних станів
    
    Вузли: регіони
    Ребра: зв'язки між регіонами з ВАГАМИ (вага = відстань + енергетичний потік)
    Атрибути: стан кожного регіону
    """
    G = nx.Graph()
    
    # Створюємо словник станів для швидкого доступу
    states_by_region = {s["region"]: s for s in canonical_states.values()}
    
    # Додаємо вузли (регіони)
    for key, state in canonical_states.items():
        region = state["region"]
        
        # Обчислюємо критичність регіону
        criticality = calculate_region_criticality(state)
        
        G.add_node(
            region,
            node_type="region",
            criticality=criticality,
            blackout_status=state["blackout_status"],
            alert_active=state["air_raid_alert"] > 0,
            demand=state["demand"],
            generation=state["net_generation"],
            data_completeness=state["data_completeness"]
        )
    
    # Додаємо ребра між сусідніми регіонами з ВАГАМИ
    for i, region1 in enumerate(CANONICAL_REGIONS):
        if region1 not in G:
            continue
        
        state1 = states_by_region.get(region1)
        if not state1:
            continue
        
        for region2 in CANONICAL_REGIONS[i+1:]:
            if region2 not in G:
                continue
            
            state2 = states_by_region.get(region2)
            if not state2:
                continue
            
            # Обчислюємо відстань між регіонами
            if region1 in REGION_COORDS and region2 in REGION_COORDS:
                dist_km = haversine_distance(
                    REGION_COORDS[region1],
                    REGION_COORDS[region2]
                )
                
                # Додаємо ребро якщо регіони близькі (< 300 км)
                if dist_km < 300:
                    # Обчислюємо вагу ребра (комбінація відстані та енергетичного потоку)
                    weight = calculate_edge_weight(
                        state1, state2, dist_km
                    )
                    
                    # Обчислюємо інтенсивність зв'язку (обернено пропорційна відстані)
                    connection_intensity = 1.0 / (1.0 + dist_km / 100.0)
                    
                    # Енергетичний потік (якщо є дані про interchange)
                    energy_flow = calculate_energy_flow(state1, state2)
                    
                    G.add_edge(
                        region1,
                        region2,
                        distance_km=dist_km,
                        weight=weight,  # ВАГА РЕБРА (для алгоритмів графів)
                        connection_intensity=connection_intensity,
                        energy_flow=energy_flow,
                        edge_type="geographic"
                    )
    
    return G


def calculate_edge_weight(
    state1: CanonicalState,
    state2: CanonicalState,
    distance_km: float
) -> float:
    """
    Обчислює вагу ребра між двома регіонами
    
    Вага враховує:
    - Географічну відстань (більша відстань = більша вага)
    - Енергетичний потік (більший потік = менша вага, тобто сильніший зв'язок)
    - Критичність регіонів (критичні регіони мають меншу вагу для швидшого поширення)
    
    Returns:
        Вага ребра (більша вага = слабший зв'язок)
    """
    # Базова вага від відстані (нормалізуємо до 0-1)
    base_weight = min(distance_km / 300.0, 1.0)
    
    # Корекція від енергетичного потоку
    flow_factor = 1.0
    if state1.get("total_interchange") and state2.get("total_interchange"):
        # Якщо є значний потік між регіонами - зменшуємо вагу
        flow_magnitude = abs(state1["total_interchange"]) + abs(state2["total_interchange"])
        if flow_magnitude > 1000:  # Значний потік
            flow_factor = 0.7  # Зменшуємо вагу на 30%
    
    # Корекція від критичності (критичні регіони мають пріоритет)
    criticality1 = calculate_region_criticality(state1)
    criticality2 = calculate_region_criticality(state2)
    avg_criticality = (criticality1 + criticality2) / 2.0
    
    # Якщо середня критичність висока - зменшуємо вагу (сильніший зв'язок)
    criticality_factor = 1.0 - (avg_criticality * 0.3)
    
    # Фінальна вага
    final_weight = base_weight * flow_factor * criticality_factor
    
    return max(final_weight, 0.1)  # Мінімальна вага 0.1


def calculate_energy_flow(
    state1: CanonicalState,
    state2: CanonicalState
) -> float:
    """
    Обчислює енергетичний потік між двома регіонами
    
    Returns:
        Величина потоку (0.0 - 1.0)
    """
    if not state1.get("total_interchange") or not state2.get("total_interchange"):
        return 0.0
    
    # Нормалізуємо потік
    flow1 = abs(state1["total_interchange"])
    flow2 = abs(state2["total_interchange"])
    
    # Середній потік нормалізуємо до 0-1
    avg_flow = (flow1 + flow2) / 2.0
    normalized_flow = min(avg_flow / 5000.0, 1.0)  # 5000 як максимальний потік
    
    return normalized_flow


def haversine_distance(coord1: tuple, coord2: tuple) -> float:
    """Обчислює відстань між двома координатами (км)"""
    from math import radians, sin, cos, sqrt, atan2
    
    lat1, lon1 = radians(coord1[0]), radians(coord1[1])
    lat2, lon2 = radians(coord2[0]), radians(coord2[1])
    
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * atan2(sqrt(a), sqrt(1-a))
    
    R = 6371  # Радіус Землі в км
    return R * c


def calculate_region_criticality(state: CanonicalState) -> float:
    """
    Обчислює критичність регіону (0.0 - 1.0)
    """
    score = 0.0
    
    # Фактор 1: Тривоги
    if state["air_raid_alert"] > 0:
        score += 0.3
    
    # Фактор 2: Відключення
    if state["blackout_status"] != "none":
        score += 0.4
        score += min(state["blackout_probability"], 0.2)
    
    # Фактор 3: Екстремальна погода
    if state["weather_extreme_flag"]:
        score += 0.2
    
    # Фактор 4: Дисбаланс енергії
    if state["demand"] and state["net_generation"]:
        imbalance = abs(state["demand"] - state["net_generation"]) / max(state["demand"], 1)
        score += min(imbalance * 0.1, 0.1)
    
    return min(score, 1.0)

print("Функції побудови графа готові")


In [ ]:
def should_call_llm(
    canonical_states: Dict[str, CanonicalState],
    energy_graph: nx.Graph
) -> Tuple[bool, List[str], Dict[str, Any]]:
    """
    Визначає чи потрібно викликати LLM та формує контекст
    ВИКОРИСТОВУЄ ВАГИ РЕБЕР для аналізу поширення критичності по графу
    
    Returns:
        (should_call, critical_regions, context)
    """
    critical_regions = []
    context = {
        "summary": {},
        "critical_events": [],
        "graph_stats": {},
        "graph_metrics": {}  # Метрики графа з урахуванням ваг
    }
    
    # Аналізуємо критичність регіонів
    for key, state in canonical_states.items():
        criticality = calculate_region_criticality(state)
        
        if criticality > 0.5:  # Поріг критичності
            critical_regions.append(state["region"])
            context["critical_events"].append({
                "region": state["region"],
                "criticality": criticality,
                "issues": [
                    f"Тривоги: {state['air_raid_alert']}" if state["air_raid_alert"] > 0 else None,
                    f"Відключення: {state['blackout_status']}" if state["blackout_status"] != "none" else None,
                    "Екстремальна погода" if state["weather_extreme_flag"] else None
                ]
            })
    
    # АНАЛІЗ ГРАФА З ВАГАМИ РЕБЕР
    if energy_graph and energy_graph.number_of_edges() > 0:
        # 1. Обчислюємо найкоротші шляхи з урахуванням ваг
        shortest_paths_analysis = analyze_shortest_paths(energy_graph, critical_regions)
        
        # 2. Обчислюємо центральність з урахуванням ваг
        centrality_metrics = calculate_weighted_centrality(energy_graph)
        
        # 3. Аналізуємо поширення критичності по графу
        criticality_propagation = analyze_criticality_propagation(
            energy_graph, canonical_states
        )
        
        # 4. Знаходимо найбільш вразливі регіони (висока центральність + низька вага зв'язків)
        vulnerable_regions = find_vulnerable_regions(
            energy_graph, centrality_metrics
        )
        
        context["graph_stats"] = {
            "total_nodes": energy_graph.number_of_nodes(),
            "total_edges": energy_graph.number_of_edges(),
            "critical_nodes": len([n for n, d in energy_graph.nodes(data=True) if d.get("criticality", 0) > 0.5]),
            "average_edge_weight": np.mean([
                d.get("weight", 1.0) for u, v, d in energy_graph.edges(data=True)
            ])
        }
        
        context["graph_metrics"] = {
            "shortest_paths": shortest_paths_analysis,
            "centrality": centrality_metrics,
            "criticality_propagation": criticality_propagation,
            "vulnerable_regions": vulnerable_regions
        }
    else:
        context["graph_stats"] = {
            "total_nodes": energy_graph.number_of_nodes() if energy_graph else 0,
            "total_edges": 0,
            "critical_nodes": 0,
            "average_edge_weight": 0.0
        }
    
    # Загальна статистика
    total_regions = len(canonical_states)
    context["summary"] = {
        "total_regions": total_regions,
        "critical_regions_count": len(critical_regions),
        "average_criticality": np.mean([calculate_region_criticality(s) for s in canonical_states.values()]),
        "data_completeness": np.mean([s["data_completeness"] for s in canonical_states.values()])
    }
    
    # ВИРІШЕННЯ ЧИ ВИКЛИКАТИ LLM з урахуванням ваг ребер
    should_call = determine_llm_call_with_weights(
        critical_regions,
        context,
        energy_graph
    )
    
    return should_call, critical_regions, context


def analyze_shortest_paths(
    graph: nx.Graph,
    critical_regions: List[str]
) -> Dict[str, Any]:
    """
    Аналізує найкоротші шляхи від критичних регіонів до інших
    Використовує ваги ребер (weight)
    """
    if not critical_regions or graph.number_of_nodes() == 0:
        return {"max_path_length": 0, "affected_regions": []}
    
    # Знаходимо найкоротші шляхи від кожного критичного регіону
    all_paths = []
    affected_regions = set()
    
    for critical_region in critical_regions:
        if critical_region not in graph:
            continue
        
        # Використовуємо Dijkstra з вагами
        try:
            paths = nx.single_source_dijkstra_path_length(
                graph, critical_region, weight="weight"
            )
            
            # Регіони на відстані < 2.0 (з урахуванням ваг)
            for region, path_length in paths.items():
                if path_length < 2.0 and region != critical_region:
                    affected_regions.add(region)
                    all_paths.append({
                        "from": critical_region,
                        "to": region,
                        "weighted_distance": path_length
                    })
        except Exception as e:
            print(f"Помилка обчислення шляхів для {critical_region}: {e}")
    
    max_path_length = max([p["weighted_distance"] for p in all_paths]) if all_paths else 0.0
    
    return {
        "max_path_length": max_path_length,
        "affected_regions": list(affected_regions),
        "paths_count": len(all_paths)
    }


def calculate_weighted_centrality(graph: nx.Graph) -> Dict[str, Dict[str, float]]:
    """
    Обчислює центральність вузлів з урахуванням ваг ребер
    """
    if graph.number_of_nodes() == 0:
        return {}
    
    centrality = {}
    
    try:
        # Betweenness centrality з вагами (вага = відстань)
        betweenness = nx.betweenness_centrality(graph, weight="weight")
        
        # Closeness centrality з вагами
        closeness = nx.closeness_centrality(graph, distance="weight")
        
        # Degree centrality (проста, без ваг)
        degree = nx.degree_centrality(graph)
        
        for node in graph.nodes():
            centrality[node] = {
                "betweenness": betweenness.get(node, 0.0),
                "closeness": closeness.get(node, 0.0),
                "degree": degree.get(node, 0.0),
                "combined": (
                    betweenness.get(node, 0.0) * 0.4 +
                    closeness.get(node, 0.0) * 0.4 +
                    degree.get(node, 0.0) * 0.2
                )
            }
    except Exception as e:
        print(f"Помилка обчислення центральності: {e}")
        # Fallback: проста центральність без ваг
        degree = nx.degree_centrality(graph)
        for node in graph.nodes():
            centrality[node] = {
                "betweenness": 0.0,
                "closeness": 0.0,
                "degree": degree.get(node, 0.0),
                "combined": degree.get(node, 0.0)
            }
    
    return centrality


def analyze_criticality_propagation(
    graph: nx.Graph,
    canonical_states: Dict[str, CanonicalState]
) -> Dict[str, Any]:
    """
    Аналізує як критичність поширюється по графу з урахуванням ваг ребер
    """
    if graph.number_of_edges() == 0:
        return {"propagation_score": 0.0, "risk_zones": []}
    
    states_by_region = {s["region"]: s for s in canonical_states.values()}
    risk_zones = []
    total_propagation = 0.0
    
    # Для кожного критичного регіону обчислюємо зону ризику
    for node, data in graph.nodes(data=True):
        criticality = data.get("criticality", 0.0)
        
        if criticality > 0.3:  # Регіон з підвищеною критичністю
            # Знаходимо сусідів та обчислюємо вплив через ваги ребер
            neighbors_risk = []
            
            for neighbor in graph.neighbors(node):
                edge_data = graph.get_edge_data(node, neighbor)
                weight = edge_data.get("weight", 1.0)
                
                neighbor_criticality = graph.nodes[neighbor].get("criticality", 0.0)
                
                # Вплив зменшується зі збільшенням ваги (відстані)
                influence = neighbor_criticality / (1.0 + weight)
                neighbors_risk.append({
                    "neighbor": neighbor,
                    "influence": influence,
                    "edge_weight": weight
                })
            
            # Сумарний ризик поширення
            propagation_score = criticality + sum(n["influence"] for n in neighbors_risk)
            
            risk_zones.append({
                "region": node,
                "base_criticality": criticality,
                "propagation_score": propagation_score,
                "neighbors_count": len(neighbors_risk)
            })
            
            total_propagation += propagation_score
    
    avg_propagation = total_propagation / len(risk_zones) if risk_zones else 0.0
    
    return {
        "propagation_score": avg_propagation,
        "risk_zones": risk_zones[:10],  # Топ-10 зон ризику
        "total_risk_zones": len(risk_zones)
    }


def find_vulnerable_regions(
    graph: nx.Graph,
    centrality_metrics: Dict[str, Dict[str, float]]
) -> List[Dict[str, Any]]:
    """
    Знаходить найбільш вразливі регіони:
    - Висока центральність (важливі для системи)
    - Низька середня вага зв'язків (легко поширюється проблема)
    """
    vulnerable = []
    
    for node in graph.nodes():
        centrality = centrality_metrics.get(node, {})
        combined_centrality = centrality.get("combined", 0.0)
        
        # Обчислюємо середню вагу зв'язків цього вузла
        edges = list(graph.edges(node, data=True))
        if edges:
            avg_edge_weight = np.mean([
                d.get("weight", 1.0) for u, v, d in edges
            ])
            
            # Вразливість = висока центральність + низька вага зв'язків
            vulnerability = combined_centrality * (1.0 / (1.0 + avg_edge_weight))
            
            vulnerable.append({
                "region": node,
                "centrality": combined_centrality,
                "avg_edge_weight": avg_edge_weight,
                "vulnerability": vulnerability
            })
    
    # Сортуємо за вразливістю
    vulnerable.sort(key=lambda x: x["vulnerability"], reverse=True)
    
    return vulnerable[:10]  # Топ-10 найбільш вразливих


def determine_llm_call_with_weights(
    critical_regions: List[str],
    context: Dict[str, Any],
    energy_graph: nx.Graph
) -> bool:
    """
    Визначає чи викликати LLM з урахуванням ваг ребер графа
    """
    # Базові умови
    basic_conditions = (
        len(critical_regions) > 0 or
        context["summary"]["average_criticality"] > 0.3 or
        context["summary"]["data_completeness"] < 0.7
    )
    
    if not basic_conditions:
        return False
    
    # Додаткові умови з урахуванням ваг ребер
    graph_metrics = context.get("graph_metrics", {})
    
    # Умова 1: Поширення критичності по графу
    propagation = graph_metrics.get("criticality_propagation", {})
    if propagation.get("propagation_score", 0.0) > 0.5:
        return True
    
    # Умова 2: Велика кількість регіонів під впливом критичних
    shortest_paths = graph_metrics.get("shortest_paths", {})
    if shortest_paths.get("affected_regions", []):
        affected_count = len(shortest_paths["affected_regions"])
        if affected_count > len(critical_regions) * 2:  # Проблема поширюється
            return True
    
    # Умова 3: Високовразливі регіони
    vulnerable = graph_metrics.get("vulnerable_regions", [])
    if vulnerable:
        high_vulnerability = [v for v in vulnerable if v["vulnerability"] > 0.3]
        if len(high_vulnerability) > 2:
            return True
    
    return basic_conditions

print("Функція визначення виклику LLM готова")


In [ ]:
def load_data_sources(state: OrchestratorState) -> OrchestratorState:
    """Вузол 1: Завантаження даних з різних джерел"""
    print("[load_data_sources] Завантаження даних...")
    
    risk_df = load_risk_data()
    energy_df = load_energy_data()
    
    state["risk_data"] = {"main": risk_df}
    state["energy_data"] = {"main": energy_df}
    state["sync_timestamp"] = datetime.now().isoformat()
    
    print(f"[load_data_sources] Завантажено: {len(risk_df)} рядків ризику, {len(energy_df)} рядків енергетики")
    
    return state


In [ ]:

def synchronize_sources(state: OrchestratorState) -> OrchestratorState:
    """Вузол 2: Синхронізація даних у канонічний стан"""
    print("[synchronize_sources] Синхронізація даних...")
    
    risk_df = state["risk_data"]["main"]
    energy_df = state["energy_data"]["main"]
    
    canonical_states = synchronize_data(risk_df, energy_df)
    
    state["canonical_states"] = canonical_states
    state["regions_processed"] = list(set(s["region"] for s in canonical_states.values()))
    
    print(f"[synchronize_sources] Синхронізовано {len(canonical_states)} станів")
    
    return state


In [ ]:
def build_graph(state: OrchestratorState) -> OrchestratorState:
    """Вузол 3: Побудова energy-graph"""
    print("[build_graph] Побудова графа енергосистеми...")
    
    canonical_states = state["canonical_states"]
    energy_graph = build_energy_graph(canonical_states)
    
    state["energy_graph"] = energy_graph
    
    print(f"[build_graph] Граф побудовано: {energy_graph.number_of_nodes()} вузлів, {energy_graph.number_of_edges()} ребер")
    
    return state


In [ ]:
def decide_llm_call(state: OrchestratorState) -> OrchestratorState:
    """Вузол 4: Визначення чи потрібно викликати LLM"""
    print("[decide_llm_call] Оцінка необхідності виклику LLM...")
    
    canonical_states = state["canonical_states"]
    energy_graph = state["energy_graph"]
    
    should_call, critical_regions, context = should_call_llm(canonical_states, energy_graph)
    
    state["should_call_llm"] = should_call
    state["critical_regions"] = critical_regions
    state["llm_context"] = context
    state["decision_required"] = should_call
    
    print(f"[decide_llm_call] Виклик LLM: {'ТАК' if should_call else 'НІ'}")
    if critical_regions:
        print(f"[decide_llm_call] Критичні регіони: {', '.join(critical_regions)}")
    
    return state


In [ ]:

def call_llm_agent(state: OrchestratorState) -> OrchestratorState:
    """Вузол 5: Виклик LLM-агента з контекстом"""
    print("[call_llm_agent] Виклик LLM-агента...")
    
    context = state["llm_context"]
    canonical_states = state["canonical_states"]
    energy_graph = state["energy_graph"]
    
    # Формуємо промпт для LLM
    prompt = format_context_for_llm(context, canonical_states, energy_graph)
    
    try:
        llm = get_openai_client()
        
        # Тут можна викликати LLM з промптом
        # response = llm.invoke(prompt)
        # state["llm_response"] = response.content
        
        print("[call_llm_agent] LLM викликано (симуляція)")
        state["llm_response"] = "LLM response would be here"
        
    except Exception as e:
        print(f"[call_llm_agent] Помилка: {e}")
        state["llm_response"] = None
    
    return state


def format_context_for_llm(
    context: Dict[str, Any],
    canonical_states: Dict[str, CanonicalState],
    energy_graph: nx.Graph
) -> str:
    """Формує текстовий контекст для передачі LLM"""
    
    parts = []
    
    # Загальна інформація
    parts.append("=== СТАН ЕНЕРГОСИСТЕМИ ===")
    parts.append(f"Загальна кількість регіонів: {context['summary']['total_regions']}")
    parts.append(f"Критичних регіонів: {context['summary']['critical_regions_count']}")
    parts.append(f"Середня критичність: {context['summary']['average_criticality']:.2f}")
    parts.append(f"Повнота даних: {context['summary']['data_completeness']:.2f}")
    
    # Критичні події
    if context["critical_events"]:
        parts.append("\n=== КРИТИЧНІ ПОДІЇ ===")
        for event in context["critical_events"]:
            parts.append(f"Регіон: {event['region']}")
            parts.append(f"  Критичність: {event['criticality']:.2f}")
            issues = [i for i in event['issues'] if i]
            if issues:
                parts.append(f"  Проблеми: {', '.join(issues)}")
    
    # Статистика графа
    if context["graph_stats"]:
        parts.append("\n=== СТРУКТУРА ЕНЕРГОСИСТЕМИ ===")
        parts.append(f"Вузлів: {context['graph_stats']['total_nodes']}")
        parts.append(f"Зв'язків: {context['graph_stats']['total_edges']}")
        parts.append(f"Критичних вузлів: {context['graph_stats']['critical_nodes']}")
    
    return "\n".join(parts)

print("Функції виклику LLM готові")


In [ ]:
from langgraph.graph import StateGraph, END


orchestrator_graph = StateGraph(OrchestratorState)

# Додаємо вузли
orchestrator_graph.add_node("load_data_sources", load_data_sources)
orchestrator_graph.add_node("synchronize_sources", synchronize_sources)
orchestrator_graph.add_node("build_graph", build_graph)
orchestrator_graph.add_node("decide_llm_call", decide_llm_call)
orchestrator_graph.add_node("call_llm_agent", call_llm_agent)

# Визначаємо початкову точку
orchestrator_graph.set_entry_point("load_data_sources")

# Додаємо переходи
orchestrator_graph.add_edge("load_data_sources", "synchronize_sources")
orchestrator_graph.add_edge("synchronize_sources", "build_graph")
orchestrator_graph.add_edge("build_graph", "decide_llm_call")

# Умовний перехід: чи викликати LLM
def should_call_llm_route(state: OrchestratorState) -> str:
    if state.get("should_call_llm", False):
        return "call_llm_agent"
    return END

orchestrator_graph.add_conditional_edges(
    "decide_llm_call",
    should_call_llm_route,
    {
        "call_llm_agent": "call_llm_agent",
        END: END
    }
)

orchestrator_graph.add_edge("call_llm_agent", END)

# Компілюємо граф
orchestrator_app = orchestrator_graph.compile()

print("Оркестратор побудовано!")
print("\nСтруктура графа:")
print("load_data_sources -> synchronize_sources -> build_graph -> decide_llm_call")
print("  -> [call_llm_agent | END]")


In [ ]:
def run_orchestrator(target_timestamp: Optional[datetime] = None) -> OrchestratorState:
    """Запускає оркестратор з вхідними даними"""
    
    initial_state: OrchestratorState = {
        "risk_data": {},
        "energy_data": {},
        "canonical_states": {},
        "energy_graph": None,
        "llm_context": None,
        "should_call_llm": False,
        "llm_response": None,
        "sync_timestamp": datetime.now().isoformat(),
        "regions_processed": [],
        "sync_errors": [],
        "decision_required": False,
        "critical_regions": []
    }
 
    
    final_state = orchestrator_app.invoke(initial_state)
 
    print(f"Оброблено регіонів: {len(final_state['regions_processed'])}")
    print(f"Критичних регіонів: {len(final_state['critical_regions'])}")
    print(f"Виклик LLM: {'ТАК' if final_state['should_call_llm'] else 'НІ'}")
    
    if final_state["llm_context"]:
        print(f"\nКонтекст для LLM:")
        print(f"  Середня критичність: {final_state['llm_context']['summary']['average_criticality']:.2f}")
        print(f"  Повнота даних: {final_state['llm_context']['summary']['data_completeness']:.2f}")
    
    return final_state

# Запускаємо оркестратор
result = run_orchestrator()


## Використання ваг ребер у визначенні виклику LLM

### Математичні операції з вагами:

1. **Найкоротші шляхи (Dijkstra)**:
   - Використовує `weight` як відстань
   - Знаходить регіони під впливом критичних
   - `nx.single_source_dijkstra_path_length(graph, node, weight="weight")`

2. **Центральність з вагами**:
   - **Betweenness centrality**: скільки найкоротших шляхів проходить через вузол
   - **Closeness centrality**: обернена сума найкоротших відстаней до всіх інших вузлів
   - Використовує ваги для обчислення відстаней

3. **Поширення критичності**:
   - Вплив критичного регіону на сусідів зменшується зі збільшенням ваги ребра
   - `influence = neighbor_criticality / (1.0 + weight)`

4. **Вразливість регіонів**:
   - Комбінація високої центральності та низької ваги зв'язків
   - `vulnerability = centrality * (1.0 / (1.0 + avg_edge_weight))`

### Умови виклику LLM з вагами:

- Поширення критичності > 0.5 (проблема поширюється по графу)
- Багато регіонів під впливом критичних (через найкоротші шляхи)
- Високовразливі регіони (важливі вузли з низькою вагою зв'язків)


In [ ]:

if 'result' in locals() and result.get("energy_graph"):
    graph = result["energy_graph"]
  
    
    if graph.number_of_edges() > 0:
        edges_with_weights = list(graph.edges(data=True))
        
        print(f"\nЗагальна кількість ребер: {len(edges_with_weights)}")
        print(f"\nСередня вага ребра: {np.mean([d.get('weight', 1.0) for u, v, d in edges_with_weights]):.3f}")
        print(f"Мінімальна вага: {min([d.get('weight', 1.0) for u, v, d in edges_with_weights]):.3f}")
        print(f"Максимальна вага: {max([d.get('weight', 1.0) for u, v, d in edges_with_weights]):.3f}")
        
        print("\nПриклади ребер з вагами:")
        for u, v, d in edges_with_weights[:5]:
            print(f"  {u} <-> {v}: вага={d.get('weight', 1.0):.3f}, "
                  f"відстань={d.get('distance_km', 0):.1f}км, "
                  f"потік={d.get('energy_flow', 0.0):.3f}")
        
        # Метрики графа
        if result.get("graph_metrics"):
            metrics = result["graph_metrics"]
            
        
            
            # Найкоротші шляхи
            if "shortest_paths" in metrics:
                sp = metrics["shortest_paths"]
                print(f"\nНайкоротші шляхи:")
                print(f"  Максимальна відстань: {sp.get('max_path_length', 0):.3f}")
                print(f"  Регіонів під впливом: {len(sp.get('affected_regions', []))}")
            
            # Центральність
            if "centrality" in metrics:
                centrality = metrics["centrality"]
                top_central = sorted(
                    centrality.items(),
                    key=lambda x: x[1].get("combined", 0.0),
                    reverse=True
                )[:5]
                print(f"\nТоп-5 найбільш центральних регіонів:")
                for region, metrics_dict in top_central:
                    print(f"  {region}: combined={metrics_dict['combined']:.3f}, "
                          f"betweenness={metrics_dict['betweenness']:.3f}")
            
            # Поширення критичності
            if "criticality_propagation" in metrics:
                prop = metrics["criticality_propagation"]
                print(f"\nПоширення критичності:")
                print(f"  Середній бал поширення: {prop.get('propagation_score', 0):.3f}")
                print(f"  Зон ризику: {prop.get('total_risk_zones', 0)}")
            
            # Вразливі регіони
            if "vulnerable_regions" in metrics:
                vuln = metrics["vulnerable_regions"]
                print(f"\nТоп-5 найбільш вразливих регіонів:")
                for v in vuln[:5]:
                    print(f"  {v['region']}: вразливість={v['vulnerability']:.3f}, "
                          f"центральність={v['centrality']:.3f}, "
                          f"середня вага={v['avg_edge_weight']:.3f}")
    else:
        print("Граф не містить ребер")
